In [7]:
import os 
import instructor
import requests
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI
from typing import List
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [8]:
load_dotenv()
client = instructor.patch(OpenAI(api_key=os.environ["OPENAI_API_KEY"]))

In [9]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/workspaces/New-Oracle/docs/pdf/ZAKON_o_davcnem_postopku_(ZDavP-2).pdf")
pages = loader.load_and_split()

In [10]:
pages[0].page_content = pages[0].page_content.replace('\n', '')
for page in pages:
    page.page_content = page.page_content.replace('\n', '')


In [12]:
pages
long_string = ''.join([page.page_content for page in pages])
print(len(long_string))

682025


In [15]:
with open('long_string.csv', 'w') as file:
    file.write(long_string)


In [32]:
import random

random.seed(42)
relevant_documents = random.sample(pages, 8)
irrelevant_documents = list(set(str(doc) for doc in pages) - set(str(doc) for doc in relevant_documents))



In [33]:
from pydantic import BaseModel, Field
from typing import List

class QueryForPassage(BaseModel):
    hypothetical_questions: List[str] = Field(
        default_factory=list,
        description="Širok spekter hipotetičnih vprašanj, na katera bi ta dokument lahko odgovoril. Vprašanja morajo biti slovnično pravilna z pravilnimi presledni in pravilno uporabo šumnikov. Namesto čudnih znakov uporabi šumnike",
    )
    hypothetical_queries: List[str] = Field(
        default_factory=list,
        description="Širok spekter hipotetičnih poizvedb, za katere bi bil ta dokument relevanten, v kontekstu iskalnika ali procesa pridobivanja informacij.Vprašanja morajo biti slovnično pravilna z pravilnimi presledni in pravilno uporabo šumnikov. Namesto čudnih znakov uporabi šumnike",
    )

In [34]:
len(relevant_documents)

8

In [35]:
candidate_queries = []
for doc in relevant_documents:
    candidate = client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=QueryForPassage,
        messages=[
            {
                "role": "system",
                "content": """Ste strokovnjak AI, ki nam pomaga ustvarjati visokokakovosten, raznolik sintetičen nabor podatkov za usposabljanje modelov za pridobivanje informacij.
                Vaša vloga je analizirati dan kos dokumenta in nam zagotoviti visokokakovostna potencialna vprašanja. Vsa vprašanja morajo biti v slovenskem jeziku.""",
            },
            {"role": "user", "content": doc.page_content},
        ],
    )
    candidate_queries.append(candidate)
    print(doc.page_content)
    print(candidate)
    

print("Document: ")
print(relevant_documents[0].page_content)
print("Generated queries: ")
candidate_queries[0].model_dump()

Ne glede na 111. in 113. člen tega zakona se zavarovanje izpoln itve ali zavarovanje plačila davčne obveznosti ne zahteva, če je zavezanec za davek neposredni uporabnik državnega proračuna v skladu s predpisom, ki ureja javne finance.  117. člen (vrste instrumentov zavarovanja)  (1) Instrumenti zavarovanja iz 1. točke pr vega odstavka 114. člena tega zakona so: 1. bančna garancija brez ugovora na prvi poziv, ki jo izda banka, ki jo kot garanta prizna davčni organ;  2. garantno pismo zavarovalnice, ki ga izda zavarovalnica, ki jo kot garanta prizna davčni organ;  3. cirkulirani certif icirani ček, če je trasat takega čeka banka, ki jo kot garanta prizna davčni organ;  4. avalirana menica, če jo je avalirala banka, ki jo kot garanta prizna davčni organ;  5. gotovinski polog.  (2) Davčni organ lahko, na zahtevo zavezanca za davek, sprejme tudi dru ge instrumente zavarovanja, če ti instrumenti na enakovreden način zagotavljajo izpolnitev oziroma plačilo davčne obveznosti.  (3) Davčni organ

{'hypothetical_questions': ['Kateri so primeri instrumentov zavarovanja, ki jih priznava davčni organ?',
  'Kdaj davčni organ ne zahteva zavarovanja izpolnitve ali plačila davčne obveznosti?',
  'Na kakšen način lahko zavezanec za davek zagotovi zavarovanje davčne obveznosti?',
  'Kakšen postopek davčni organ uporabi za pridobitev zastavne pravice?',
  'Kaj pomeni predznambo zastavne pravice v kontekstu davčnega zavarovanja?',
  'Kako lahko davčni organ zavrne predlagani instrument zavarovanja?',
  'Kakšen je rok za davčni organ, da predlaga vknjižbo zastavne pravice?'],
 'hypothetical_queries': ['vrste instrumentov zavarovanja po slovenskem davčnem zakonu',
  'izjeme pri zahtevah za zavarovanje davkov za državne uporabnike',
  'postopek vpisa zastavne pravice na deležu družbenika v družbi',
  'predznambo zastavne pravice in njeno vknjižbo',
  'rok za vknjižbo zastavne pravice po sklepu davčnega organa',
  'zavrnitev instrumenta zavarovanja s strani davčnega organa',
  'postopek zavaro

In [36]:
candidate_queries

[QueryForPassage(hypothetical_questions=['Kateri so primeri instrumentov zavarovanja, ki jih priznava davčni organ?', 'Kdaj davčni organ ne zahteva zavarovanja izpolnitve ali plačila davčne obveznosti?', 'Na kakšen način lahko zavezanec za davek zagotovi zavarovanje davčne obveznosti?', 'Kakšen postopek davčni organ uporabi za pridobitev zastavne pravice?', 'Kaj pomeni predznambo zastavne pravice v kontekstu davčnega zavarovanja?', 'Kako lahko davčni organ zavrne predlagani instrument zavarovanja?', 'Kakšen je rok za davčni organ, da predlaga vknjižbo zastavne pravice?'], hypothetical_queries=['vrste instrumentov zavarovanja po slovenskem davčnem zakonu', 'izjeme pri zahtevah za zavarovanje davkov za državne uporabnike', 'postopek vpisa zastavne pravice na deležu družbenika v družbi', 'predznambo zastavne pravice in njeno vknjižbo', 'rok za vknjižbo zastavne pravice po sklepu davčnega organa', 'zavrnitev instrumenta zavarovanja s strani davčnega organa', 'postopek zavarovanja dolga pri

In [37]:
pairs = []
num_questions = 6
num_queries = 6
random.seed(42)

for candidates, doc in zip(candidate_queries, relevant_documents):
    candidates = candidates.model_dump()
    queries = random.sample(candidates['hypothetical_questions'], num_questions)
    queries += random.sample(candidates['hypothetical_queries'], num_queries)
    for q in queries:
        pairs.append([q, doc.page_content])

In [39]:
import pandas as pd
len(pairs)
pairs
# Convert the pairs list to a DataFrame
df = pd.DataFrame(pairs)

# Save the DataFrame to a CSV file
df.to_csv('pairs.csv', index=False)



In [1]:
import pandas as pd

pairs_list = pd.read_csv('pairs.csv')
pairs = pairs_list.values.tolist()




In [2]:
pairs

[['Kako lahko davčni organ zavrne predlagani instrument zavarovanja?',
  'Ne glede na 111. in 113. člen tega zakona se zavarovanje izpoln itve ali zavarovanje plačila davčne obveznosti ne zahteva, če je zavezanec za davek neposredni uporabnik državnega proračuna v skladu s predpisom, ki ureja javne finance.  117. člen (vrste instrumentov zavarovanja)  (1) Instrumenti zavarovanja iz 1. točke pr vega odstavka 114. člena tega zakona so: 1. bančna garancija brez ugovora na prvi poziv, ki jo izda banka, ki jo kot garanta prizna davčni organ;  2. garantno pismo zavarovalnice, ki ga izda zavarovalnica, ki jo kot garanta prizna davčni organ;  3. cirkulirani certif icirani ček, če je trasat takega čeka banka, ki jo kot garanta prizna davčni organ;  4. avalirana menica, če jo je avalirala banka, ki jo kot garanta prizna davčni organ;  5. gotovinski polog.  (2) Davčni organ lahko, na zahtevo zavezanca za davek, sprejme tudi dru ge instrumente zavarovanja, če ti instrumenti na enakovreden način za

In [3]:
from ragatouille import RAGTrainer


trainer = RAGTrainer(model_name="tax", pretrained_model_name="colbert-ir/colbertv2.0")


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
print(torch.__version__)
import torch
x = torch.rand(5, 3)
print(x)


torch.cpu.is_available()

2.2.2+cu118
tensor([[3.8840e-01, 4.8901e-01, 1.1588e-01],
        [2.7439e-01, 9.4736e-04, 9.5023e-01],
        [4.7329e-02, 1.4746e-01, 2.8838e-02],
        [3.9673e-01, 5.8177e-02, 2.6355e-01],
        [1.1253e-01, 6.3368e-01, 6.5979e-01]])


True

In [5]:
print(pairs)
print(long_string)

[['Kako lahko davčni organ zavrne predlagani instrument zavarovanja?', 'Ne glede na 111. in 113. člen tega zakona se zavarovanje izpoln itve ali zavarovanje plačila davčne obveznosti ne zahteva, če je zavezanec za davek neposredni uporabnik državnega proračuna v skladu s predpisom, ki ureja javne finance.  117. člen (vrste instrumentov zavarovanja)  (1) Instrumenti zavarovanja iz 1. točke pr vega odstavka 114. člena tega zakona so: 1. bančna garancija brez ugovora na prvi poziv, ki jo izda banka, ki jo kot garanta prizna davčni organ;  2. garantno pismo zavarovalnice, ki ga izda zavarovalnica, ki jo kot garanta prizna davčni organ;  3. cirkulirani certif icirani ček, če je trasat takega čeka banka, ki jo kot garanta prizna davčni organ;  4. avalirana menica, če jo je avalirala banka, ki jo kot garanta prizna davčni organ;  5. gotovinski polog.  (2) Davčni organ lahko, na zahtevo zavezanca za davek, sprejme tudi dru ge instrumente zavarovanja, če ti instrumenti na enakovreden način zago

NameError: name 'long_string' is not defined

In [53]:

trainer.prepare_training_data(raw_data=pairs, all_documents=long_string , data_out_path='./data/', num_new_negatives=10, hard_negative_minimum_rank=10, mine_hard_negatives=True, hard_negative_model_size='small', pairs_with_labels=False, positive_label=1, negative_label=0)

Loading Hard Negative SimpleMiner dense embedding model BAAI/bge-small-en-v1.5...
Building hard negative index for 84 documents...
All documents embedded, now adding to index...
save_index set to False, skipping saving hard negative index
Hard negative index generated


'./data/'

In [6]:
trainer.train(batch_size=16,
              nbits=2, # How many bits will the trained model use when compressing indexes
              maxsteps=100000, # Maximum steps hard stop
              use_ib_negatives=True, # Use in-batch negative to calculate loss
              dim=128, # How many dimensions per embedding. 128 is the default and works well.
              learning_rate=5e-6, # Learning rate, small values ([3e-6,3e-5] work best if the base model is BERT-like, 5e-6 is often the sweet spot)
              doc_maxlen=256, # Maximum document length. Because of how ColBERT works, smaller chunks (128-256) work very well.
              use_relu=False, # Disable ReLU -- doesn't improve performance
              warmup_steps="auto", # Defaults to 10%
             )

AttributeError: 'RAGTrainer' object has no attribute 'data_dir'